<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
</div>  

<br><br><br><br>

# Predicting dose-response curves from ligand kinetic values

In this tutorial we will simulate mathematical model of a signaling pathway to obtain dose-response curves, and consequently, predict the *efficacy (EC$_{50}$)* of drugs. 




This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/SSBtoolkit-Tutorial3A.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="70" style="margin:20px">
</a>
</div> 

## Setup

To run SSBtoolkit within Google Colab, you'll need to run the following installation commands. You can of course run this tutorial locally if you prefer.


<span style="color:black"> ⚠️ WARNING: this notebook is prepared to run on linux machines. If you intend to run it <b>locally</b> on a different OS (MacOS or Windows) you have to set the BioNetGen environment path manually. Running directly on Google colab is OS independent.</span>

# Install Dependencies


In [ ]:
#@title Install dependencies
%%bash

# download SSB source code and install dependencies
if [ ! -d "SSBtoolkit/" ]; then
  git clone https://github.com/rribeiro-sci/SSBtoolkit.git --quiet
  pip install -r SSBtoolkit/requirements.txt
fi


In [ ]:
cd SSBtoolkit

In [1]:
#Import Python dependencies
import os, sys, json, math, site
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pubchempy import *
from sklearn.linear_model import LinearRegression

from src.lib.ssbtoolkit import convert, get, simulation

In [2]:
#Setting BioNetGen environment path
distpath = site.getsitepackages()[0]
BioNetGen=os.path.join(distpath, 'bionetgen/bng-linux:')
mypath=%env PATH
newpath=BioNetGen+mypath
%env PATH=$newpath

env: PATH=/usr/local/lib/python3.8/dist-packages/bionetgen/bng-linux:/sbin:/usr/sbin:/bin:/usr/bin:/usr/local/bin


In [3]:
#Test bioservices
from bioservices import UniProt
u = UniProt(verbose=False)

# Loading experimental Data

Once the SSBtoolkit environment is set up we are ready to start to simulate.

We will begin by load the kinetic data of some ligands of the Adenosine 2A receptor. This data was taken from *([Guo, D. et al., 2017](https://pubs.acs.org/doi/10.1021/acs.chemrev.6b00025))*. This data can be found in `examples/A2AR_Kinetic_values_example.csv`

In [4]:
data = pd.read_csv('example/A2AR_Kinetic_values_example.csv')
data

,cmpd,target,T (°C),kon (1/(uM*s)),koff (1/s),kd (nM)
0,Neca,HEK293hA2AR membranes,4,114.0,3.180,28.000
1,UK432097,HEK293hA2AR membranes,5,300.0,0.240,8.000
2,LUF5835,HEK293hA2AR membranes,5,960.0,17.400,18.000
3,ZM241385,HEK293hA2AR membranes,4,7800.0,0.840,0.110
4,LUF6805,HEK293hA2AR membranes,4,12000.0,21.000,1.800
5,LUF6632,HEK293hA2AR membranes,4,2040.0,0.186,0.091


Commonly the kinetic parameters, k<sub>on</sub> and k<sub>off</sub>, are measured experimentally at low temperatures in order to slow down the reaction so it can be measured. Therefore, we will use the SSBtoolkit built-in function `convert.KineticTempScale()` to scale the kinetic parameters to room temperature (25 °C). This is achieved using the free energy equation:

$ \Delta G = -RTln(K_d)$ 

$K_d = \frac{k_{off}}{k_{on}}$

In [5]:
#We will create a new dataframe with the scaled values 
data_scaled = data.copy()
data_scaled['kon (1/(uM*s))'] = data.apply(lambda row: convert.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[0], axis=1)
data_scaled['koff (1/s)'] = data.apply(lambda row: convert.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[1], axis=1)
data_scaled['T (°C)'] = 25
data_scaled

,cmpd,target,T (°C),kon (1/(uM*s)),koff (1/s),kd (nM)
0,Neca,HEK293hA2AR membranes,25,122.638,3.421,28.000
1,UK432097,HEK293hA2AR membranes,25,321.571,0.257,8.000
2,LUF5835,HEK293hA2AR membranes,25,1029.028,18.651,18.000
3,ZM241385,HEK293hA2AR membranes,25,8391.016,0.904,0.110
4,LUF6805,HEK293hA2AR membranes,25,12909.255,22.591,1.800
5,LUF6632,HEK293hA2AR membranes,25,2194.573,0.200,0.091


# Selecting the signaling pathway 

The core of the SSBtoolkit is, naturally, the mathematical models of the signaling pathways. 

Since G-protein sub-families are classified by their $\alpha$ subunits, this classfication as been served to identify their signaling pathways:
* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 See [The SSB Framework](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/docs/ssb_framework.md) documentation for more details.

We can define manualy the G$\alpha$ pathway we want to work with, or simply query our internal database of human GPCR pathways using the UNIPROT id of our receptor. The UNIPROT id for the human Adenosine A2 receptot is [P29274](https://www.uniprot.org/uniprot/P29274).

In [6]:
uniprotID = 'P29274'
gprotein=get.gprotein(uniprotID)
gprotein

'Gs'

💡 It is also possible to known the associated G$\alpha$ pathway of a GPCR using the primary sequence, see in [Docs](https://github.com/rribeiro-sci/SSBtookit/blob/main/docs/ssb_code.md).


<span style="color:black">⚠️ WARNING: our framework was specifically design for human GPCR. The quering for pathways for GPCRs other than Human may be included in the future. However you want to study a non-human GPCR you can still use the SSB framework by setting manually the G$\alpha$ pathway.</span>

# Setting the parameters for simulation

To predict a dose-response curve from the simulation of signaling pathways, individual simulations of the pathway according to a series of ligand's concentrations must be performed (as it would be done in the wet-lab).  

To define an array of ligand concentrations we will use a geometric progression. The reason why we use a geometric progression is due to the impact of the dilution fraction on the accuracy of K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values experimentally estimated. This factor, that defines the spacing between the adjacent concentration values, has an impact on the concentration values that are on the linear portion of the curve. Therefore, using a geometric progression we can mimic the experimental conditions where each concentration equals to the power of 2 of the previous lowest concentration *([Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426))*

<span style="color:black"> ⚠️ WARNING: the SSB toolkit uses μM as default concentration units.</span>


In [7]:
# the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# defining other simulation parameters:
time = 10000  # time of simulation in seconds
dt   = 1000     # integration step for simulation


## Running the simulation

After having defined all the simulation parameters we are ready to proceed with the simulations. A simulation of a methamatical model of a signaling pathway consists of the integration of a set of ordinary differential equations (ODEs) as function of time. Since each ODE represents a molecular event of the signaling pathway, when integrated over time, such equations can describe how the concentration of species inside the model changes over time.

In the <b>Tutorial1</b>, the key point is using  the drug-receptor affinity value (K<sub>d</sub>) to fire up the model. However, in this tutorial we will use the kinetic parameters to simulate the ligand-receptor binding.

Because we are using kinetic values we have to set `kinetics=True` in the `simulation.SetSimulationParameters()` instance. If we use option `kinetcs=True`, we also need to pass to the instance a dictionary of parameters. We can see a list of all parameters that can be changed for each pathway [here](https://github.com/rribeiro-sci/SSBtoolkit/tree/main/SI).

For this tutorial we just need to include the following kinetic parameters:
* `RL_kon` for the ligand-receptor interaction forward parameter (μM<sup>-1</sup>s<sup>-1</sup>)
* `RL_koff` for the ligand-receptor interaction reverse parameter (s<sup>-1</sup>)

In [8]:
#preparing the parameters for each ligand:
kinetic_parameters=[]
for k, v in data_scaled.iterrows():
    kinetic_parameters.append({'RL_kon':v['kon (1/(uM*s))'],'RL_koff':v['koff (1/s)']})

In [9]:
#create a simulation instance.
sim = simulation.activation()

In [10]:
#Setting the simulation parameters
sim.SetSimulationParameters(ligands=data_scaled.cmpd.to_list()[1:3], 
                              pathway=gprotein, 
                              receptor_conc=receptor_conc, 
                              lig_conc_range=lig_conc_range, 
                              ttotal=time, 
                              nsteps=dt,  
                              kinetics=True, 
                              kinetic_parameters=kinetic_parameters)


In [12]:
sim.PathwayParameters()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
#Running the simulation
sim.Run()

In the end, the concentration values of the species of the signaling pathway over the simulation time will be saved inside the instance.

The response of a signaling pathway is, naturally, represented by the increase or decrease of one of the species described by the model. So, to predict the dose-response curve we need, firstly, to extract the maximum concentration value orbserved for one specie from each individual simulation (from the series of simulations for each ligand concentration). Then, such values will be fitted to a logistic regression. 

To achieve this, we will use the analysis attribute:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.Curve()

Finnaly, from the dose-response curves we can interpolate the EC<sub>50</sub> values.


In [ ]:
sim.Potency()

💡 The potency predicted values can be exported as a python dictionary usinf the attribute `sim.PotencyToDict()` or saved in a csv file using the attribute `sim.PotencyToCSV()`. 📖 See the [Docs](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/docs/ssb_code.md) to know how to save directly on Google Drive and for more details.

# Congratulations!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with SSB framework, we encourage you to finish the rest of the tutorials in this series. 

# Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{XXX,
    title={XXX},
    author={XXX},
    publisher={XXX},
    note={\url{XXX}},
    year={XXX}
}
```

# Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  